In [1]:
#imports
import string
import pandas as pd
import pickle as pkl
from collections import Counter
from matplotlib import pyplot as plt

import numpy as np
from random import sample

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

import sys
sys.path.append('../../src')
import utils

2023-07-17 03:19:25.283846: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-17 03:19:25.283900: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
history = pkl.load(open('../../data/embeddings/history_embeddings-todo', 'rb'))

In [75]:
h = np.array(h).flatten()
h.shape
max_len

2816

In [90]:
X = []
max_len = 0

for user in history:
    h = np.array(history[user]).flatten()
    if len(h) > max_len:
        max_len = len(h)

for user in history:
    h = np.array(history[user]).flatten()
    if len(h) < max_len:
        h = np.pad(h, (0, max_len - len(h)), 'constant', constant_values=(0))

    X.append(h)

X = np.array(X)

In [91]:
#train test split
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

In [93]:
X_train.shape

(1031, 2816)

In [122]:
class Autoencoder(Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
          layers.Dense(2816, activation='relu'),
          layers.Dense(1024, activation='relu'),
          layers.Dense(latent_dim, activation='tanh'),
        ])
        
        self.decoder = tf.keras.Sequential([
          layers.Dense(1024, activation='relu'),
          layers.Dense(2816, activation='tanh')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(1000)

autoencoder.compile(optimizer='adam', loss='cosine_similarity')

In [123]:
from tqdm.keras import TqdmCallback

In [ ]:
autoencoder.fit(X_train, X_train, epochs=500, shuffle=True, batch_size=16, validation_data=(X_test, X_test), verbose=0, callbacks=[TqdmCallback(verbose=1)])

In [128]:
histories = autoencoder.encoder(X)
for h, user in zip(histories, history):
    history[user] = h.numpy()

In [130]:
pkl.dump(history, open('../../data/embeddings/history_embeddings', 'wb'))